## Class Tutorial

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pickle 
from astropy.io import fits
%matplotlib widget

We use a class to define several functions in a self-consistent way repeatedly and be able to use them in different notebooks without rewriting them.

### Class Objects
Class objects support two kind of operations: *attribute references* and *instantiation*. **Attribute references** use the standard syntax in python (obj.name).

In [2]:
# Example:
class MyClass:
    """A simple example class"""
    internal_variable = 12345

    def print_hello(self):
        return 'Hello World'

# To retrieve the value of the internal variable we use:
print(MyClass.internal_variable)
# To use the function of the class we use
MyClass.print_hello(MyClass)

12345


'Hello World'

The argument $\texttt{self}$ of the class means that the function requires the class itself as an argument.

**Class instantiation** uses function notation. We can create a new instance of the class and assign his object to a local variable.

In [4]:
first_class = MyClass()
second_class = MyClass()
# We have two objects of type MyClass with all the attributes and functions
# of the original class but will work independently

second_class.internal_variable = 67890
print('first_class internal variable value:', first_class.internal_variable)
print('second_class internal variable value:', second_class.internal_variable)

first_class internal variable value: 12345
second_class internal variable value: 67890


A variable inside a class takes the name of **attribute** while functions are called **methods**. When calling a method of an instance object you don0t need to specify the self argument

In [8]:
MyClass.print_hello(MyClass) #Here I need the self argument
second_class.print_hello() #Here I don't need it, notice the difference

'Hello World'

### Class Initialization
Classes support a special kind of method that is automatically called **every time** you create a new class instance. This inizialization method is defined by the name ```__init__```. When we create a new object using our class, the init method is automatically invoked. In the following example init requires two arguments other than the self one, so we have to provide them to avoid an error 

In [9]:
class MySecondClass:
    def __init__(self, x_input, y_input):
        self.x_pos = x_input
        self.y_pos = y_input

#new_class = MySecondClass() -> WRONG! No arguments
new_class = MySecondClass(258, 76) # Correct

If we change the class, we have to make a new instance, the previous one won't be updated.

In [ ]:
class MySecondClass:
    def __init__(self, x_input, y_input):
        self.x_pos = x_input
        self.y_pos = y_input

    def print_position(self):
        print('Attributes called from a class method', self.x_pos, self.y_pos)

    def change_position(self, x_new, y_new):
        self.x_pos = x_new
        self.y_pos = y_new
